<a href="https://colab.research.google.com/github/jyryu3161/biocomputing/blob/main/lab10_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ollama를 이용한 LLM 실습

Request

함수	설명
- requests.get(url, params=None)	서버에서 데이터 가져오기
- requests.post(url, data=None, json=None)	데이터 전송 (폼 or JSON)
- requests.put(url, data=None)	데이터 수정 요청
- requests.delete(url)	데이터 삭제 요청
- requests.head(url)	헤더만 요청 (본문 제외)

In [1]:
import requests

# GET 요청
response = requests.get("https://api.github.com")

# 상태 코드 확인
print(response.status_code)   # 200 등

# 응답 본문 (문자열)
print(response.text)

# JSON 형태 응답
data = response.json()
print(data)

200
{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_s

런타임 유형 -> GPU로 변경

#### 공식 설치 스크립트를 통한 Ollama 설치

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve > ollama.log &

nohup: redirecting stderr to stdout


#### Ollama 실행

In [ ]:
!ollama list

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [ ]:
!ollama pull qwen3:30b

In [ ]:
!ollama list

NAME         ID              SIZE      MODIFIED           
qwen3:30b    e50831eb2d91    18 GB     About a minute ago    
gemma3:4b    a2af6cc3eb7f    3.3 GB    5 minutes ago         


In [ ]:
!ollama run gemma3:4b "대한민국의 수도는 어디야?"

대한민국의 수도는 서울입니다.



Colab에서는 requests를 사용해 Ollama의 로컬 REST API(http://localhost:11434/api/generate)에 요청

In [ ]:
import requests
import json

# Ollama API 엔드포인트
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# 질문 설정
payload = {
    "model": "gemma3:4b",
    "prompt": "인공지능이 약물 표적 발굴에 어떻게 활용되는지 설명해줘.",
    "stream": False
}

# 요청 보내기
response = requests.post(OLLAMA_API_URL, json=payload)

# 결과 출력
if response.status_code == 200:
    data = response.json()
    print("=== 응답 ===")
    print(data["response"])
else:
    print(f"오류 발생: {response.status_code} - {response.text}")


=== 응답 ===
<think>
Okay, the user is asking how AI is used in drug target discovery. Let me start by recalling what I know about this topic. Drug target discovery is the process of identifying molecules, like proteins, that are involved in diseases and can be targeted by drugs. Traditionally, this process is time-consuming and expensive, involving a lot of trial and error.

So, how does AI come into play here? I remember reading about machine learning models analyzing large datasets. Maybe they use genomic data, protein structures, or existing drug databases. Let me break it down step by step.

First, AI can analyze biological data. For example, using genomic and proteomic data to find which genes or proteins are linked to diseases. Machine learning algorithms can spot patterns that humans might miss. Like, if a certain protein is overexpressed in cancer cells, AI might flag it as a potential target.

Then there's protein structure prediction. I think AlphaFold by DeepMind is a big dea

# PubMed 논문 분석

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.1 MB/s eta 0:00:00


In [ ]:
# =========================
# 0) 준비: 라이브러리
# =========================
from Bio import Entrez, Medline
import os, json, time, re, requests
from typing import List, Dict
import pandas as pd
import tqdm

# =========================
# 1) 사용자 설정(여기만 바꾸면 됨)
# =========================
DISEASE = "non-small cell lung cancer"   # 검색할 질병명
RETMAX  = 10                              # 상위 몇 편 가져올지
NCBI_EMAIL   = "you@example.com"          # Entrez 필수(아무 이메일 가능)
NCBI_API_KEY = os.getenv("NCBI_API_KEY", "")  # 선택(있으면 넣기)
NCBI_DELAY_SEC = 0.4                       # 서버 과부하 방지(0.3~0.5 추천)

# 로컬 Ollama (설치돼 있어야 함: https://ollama.com)
OLLAMA_URL   = "http://localhost:11434/api/generate"
OLLAMA_MODEL = "gemma3:4b"                 # 원하는 로컬 모델 이름

# =========================
# 2) PubMed 검색/가져오기 (간단 버전)
# =========================
def search_pubmed(disease: str, retmax: int) -> List[str]:
    """질병명으로 PubMed 검색 → PMID 리스트"""
    Entrez.email = NCBI_EMAIL
    if NCBI_API_KEY:
        Entrez.api_key = NCBI_API_KEY

    query = f'("{disease}"[Title/Abstract]) AND hasabstract[text]'
    with Entrez.esearch(db="pubmed", term=query, retmax=retmax, sort="relevance") as h:
        rec = Entrez.read(h)
    pmids = rec.get("IdList", [])
    return pmids

def fetch_medline(pmids: List[str]) -> List[Dict]:
    """PMID 리스트로 MEDLINE 레코드 가져오기"""
    if not pmids:
        return []
    with Entrez.efetch(db="pubmed", id=",".join(pmids), rettype="medline", retmode="text") as h:
        records = list(Medline.parse(h))
    time.sleep(NCBI_DELAY_SEC)  # 너무 빨리 호출하지 않기
    return records

# =========================
# 3) LLM 추출 (JSON만 받기, 옵션 최소화)
# =========================
SYSTEM_PROMPT = """당신은 신약개발/약리학 전문가입니다.
아래 스키마의 JSON만 출력하세요. JSON 바깥 텍스트 금지.

JSON Schema (키만 준수; 값은 한국어):
{
  "논문 PMID": "예: 12345678",
  "논문 제목": "제목",
  "질병 이름": "입력 질병명과 일치/근접",
  "타깃 이름": "EGFR/PD-1 등(없으면 unknown)",
  "작용 기전": "한 문장 요약(없으면 unknown)",
  "임상 정보 유무": "예/아니오/need_verification"
}
"""

def build_prompt(disease: str, pmid: str, title: str, abstract: str) -> str:
    return (
        "다음은 PubMed 논문의 메타데이터와 초록입니다.\n"
        f"- 질병: {disease}\n"
        f"- PMID: {pmid}\n"
        f"- 제목: {title}\n"
        f"- 초록: {abstract}\n\n"
        "요구사항:\n"
        "1) 위 스키마의 키를 가진 순수 JSON만 출력.\n"
        "2) 모르면 'unknown' 또는 'need_verification' 사용.\n"
        "3) 임상 정보가 분명하면 '예', 불명확하면 '아니오' 또는 'need_verification'."
    )

def _safe_json_loads(text: str) -> Dict:
    """응답에서 첫 번째 JSON 객체만 안전하게 추출"""
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        m = re.search(r"\{.*\}", text, flags=re.DOTALL)
        if not m:
            raise
        return json.loads(m.group(0))

def extract_with_ollama(record: Dict, disease: str,
                        model: str = OLLAMA_MODEL,
                        url: str = OLLAMA_URL,
                        timeout: int = 120) -> Dict:
    pmid = str(record.get("PMID", ""))
    title = record.get("TI", "")
    abstract = record.get("AB", "")

    payload = {
        "model": model,
        "system": SYSTEM_PROMPT,
        "prompt": build_prompt(disease, pmid, title, abstract),
        "stream": False,
        "format": "json"  # JSON만 반환하도록 간단 지정
    }
    r = requests.post(url, json=payload, timeout=timeout)
    r.raise_for_status()
    raw = (r.json().get("response") or "").strip()
    if not raw:
        raise RuntimeError("Ollama에서 빈 응답을 받았습니다.")
    obj = _safe_json_loads(raw)

    # 누락 시 기본값 채우기
    obj["논문 PMID"] = pmid
    obj["논문 제목"] = title
    obj["질병 이름"] = disease
    obj.setdefault("타깃 이름", "unknown")
    obj.setdefault("작용 기전", "unknown")
    obj.setdefault("임상 정보 유무", "need_verification")
    return obj

# =========================
# 4) 출력/저장
# =========================
def save_csv(rows: List[Dict], filename: str = "pubmed_summary.csv"):
    df = pd.DataFrame([{
        "PMID": r.get("논문 PMID", ""),
        "제목": r.get("논문 제목", ""),
        "질병": r.get("질병 이름", ""),
        "타깃": r.get("타깃 이름", ""),
        "작용 기전": r.get("작용 기전", ""),
        "임상 정보": r.get("임상 정보 유무", "")
    } for r in rows])
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"[CSV 저장 완료] {filename}")

def pretty_print(rows: List[Dict]):
    print("\n=== 결과(JSON) ===")
    for i, r in enumerate(rows, 1):
        print(f"\n# {i}")
        print(json.dumps(r, ensure_ascii=False, indent=2))

# =========================
# 5) 실행
# =========================
if __name__ == "__main__":
    print(f"[1/3] PubMed 검색: {DISEASE!r} (최대 {RETMAX}편)")
    pmids = search_pubmed(DISEASE, RETMAX)
    if not pmids:
        raise SystemExit("검색 결과가 없습니다. DISEASE 값을 바꿔보세요.")
    print("→ PMID:", ", ".join(pmids))

    print("[2/3] MEDLINE 초록 가져오는 중...")
    records = fetch_medline(pmids)
    if not records:
        raise SystemExit("초록을 가져오지 못했습니다.")

    print("[3/3] Ollama로 JSON 추출 중...")
    results = []
    for rec in tqdm.tqdm(records):
        try:
            results.append(extract_with_ollama(rec, DISEASE))
        except Exception as e:
            print(f" - PMID {rec.get('PMID','?')} 처리 실패: {e}")

    if not results:
        raise SystemExit("추출 결과가 없습니다.")

    pretty_print(results)
    save_csv(results, "pubmed_summary.csv")
    print("\n[완료]")

[1/3] PubMed 검색: 'non-small cell lung cancer' (최대 10편)
→ PMID: 35323350, 37015337, 34985915, 36291069, 33668021, 28404761, 34150616, 38094015, 37168875, 29303290
[2/3] MEDLINE 초록 가져오는 중...
[3/3] Ollama로 JSON 추출 중...


100%|██████████| 10/10 [00:26<00:00,  2.63s/it]


=== 결과(JSON) ===

# 1
{
  "논문 PMID": "35323350",
  "논문 제목": "Non-Small-Cell Lung Cancer in 2022: A Review for General Practitioners in Oncology.",
  "질병 이름": "non-small cell lung cancer",
  "타깃 이름": "unknown",
  "작용 기전": "현재의 최적의 관행에 기반한 비소세포 폐암 치료에 대한 최신 데이터 및 관리 방법에 대한 종합 검토.",
  "임상 정보 유무": "예"
}

# 2
{
  "논문 PMID": "37015337",
  "논문 제목": "NCCN Guidelines(R) Insights: Non-Small Cell Lung Cancer, Version 2.2023.",
  "질병 이름": "non-small cell lung cancer",
  "타깃 이름": "unknown",
  "작용 기전": "eligible patients with resectable NSCLC에 대한 neoadjuvant and adjuvant systemic therapy 옵션 제시",
  "임상 정보 유무": "need_verification"
}

# 3
{
  "논문 PMID": "34985915",
  "논문 제목": "Practical Management of Oligometastatic Non-Small-Cell Lung Cancer.",
  "질병 이름": "non-small cell lung cancer",
  "타깃 이름": "unknown",
  "작용 기전": "Local ablative therapies, including surgery or stereotactic radiotherapy (SABR), are becoming an integral component in the treatment of oligometastatic disease in non-small-cell lung ca